In [6]:
import cv2
import numpy as np

#ウェブカメラからの入力の開始(標準のWEBカメラ)
cap=cv2.VideoCapture(0)
while True:
    #カメラ画像の読み込み
    _,frame=cap.read()
    #画像を縮小表示する
    frame=cv2.resize(frame,(500,300))
    #ウィンドウに画像を出力
    cv2.imshow('OpenCV Web Camera',frame)
    #ESCかEnterキーが押されたらループを抜ける
    k=cv2.waitKey(1) #1msec確認(そのとき押されているキーを入力)
    if k==27 or k==13:break
        
cap.release()#カメラを開放
cv2.destroyAllWindows()#ウィンドウを破棄

In [7]:
import cv2
import numpy as np

#ウェブカメラからの入力の開始(標準のWEBカメラ)
cap=cv2.VideoCapture(0)
while True:
    #カメラ画像の読み込み
    _,frame=cap.read()
    #画像を縮小表示する
    frame=cv2.resize(frame,(500,300))
    #青色と緑色の成分を0に(Numpyのインデックスを利用)
    frame[:,:,0]=0#青色要素を0
    frame[:,:,1]=0#緑色色素を0
    #ウィンドウに画像を出力
    cv2.imshow('RED Camera',frame)
    #エンターが押されたら抜ける
    if cv2.waitKey(1)==13:break

cap.release()#カメラを開放
cv2.destroyAllWindows()#ウィンドウを破棄

In [13]:
import cv2
import numpy as np

#ウェブカメラからの入力の開始(標準のWEBカメラ)
cap=cv2.VideoCapture(0)
while True:
    #カメラ画像の読み込み
    _,frame=cap.read()
    #画像を縮小表示する
    frame=cv2.resize(frame,(500,300))
    #色空間をHSVに変換
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV_FULL)
    #HSVを分割
    h=hsv[:,:,0]
    s=hsv[:,:,1]
    v=hsv[:,:,2]
    #赤色っぽい色を持つ画素だけを抽出
    img=np.zeros(h.shape,dtype=np.uint8)
    img[((h<50)|(h>200))&(s>100)]=255#255=白色で塗りつぶす
    #ウィンドウに画像を出力
    cv2.imshow('RED Camera',img)
    if cv2.waitKey(1)==13:break
        
cap.release()#カメラ解放
cv2.destroyAllWindows()#ウィンドウを破棄

In [15]:
import cv2

cap=cv2.VideoCapture(0)
img_last=None#前回の画像を記憶する変数
green=(0,255,0)

while True:
    #画像を取得
    _,frame=cap.read()
    frame=cv2.resize(frame,(500,300))
    #白黒画像に変換
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray,(9,9),0)
    img_b=cv2.threshold(gray,100,255,cv2.THRESH_BINARY)[1]
    #差分を確認
    if img_last is None:
        img_last=img_b
        continue
    frame_diff=cv2.absdiff(img_last,img_b)#画像ごとの差分を調べる
    cnts=cv2.findContours(frame_diff,
                         cv2.RETR_EXTERNAL,
                         cv2.CHAIN_APPROX_SIMPLE)[0]
    #差分があった点を画面に描く
    for pt in cnts:
        x,y,w,h=cv2.boundingRect(pt)
        if w<30:continue#小さな変更点は無視
        cv2.rectangle(frame,(x,y),(x+w,y+h),green,2)
    #今回のフレームを保存
    img_last=img_b
    #画面に表示
    cv2.imshow("Diff Camera",frame)
    cv2.imshow("diff data",frame_diff)
    if cv2.waitKey(1)==13: break
cap.release()
cv2.destroyAllWindows()

In [19]:
import cv2,os

img_last=None#前回の画像
no=0#画像の枚数
save_dir="./exfish"#保存ディレクトリ名
#os.mkdir(save_dir)#ディレクトリを作成

#動画ファイルから入力を開始--1
cap=cv2.VideoCapture("fish.mp4")
while True:
    #画像を取得
    is_ok,frame=cap.read()
    if not is_ok: break
    frame=cv2.resize(frame,(640,360))
    #白黒画像に変換--2
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray,(15,15),0)
    img_b=cv2.threshold(gray,127,255,cv2.THRESH_BINARY)[1]
    #差分を確認する
    if not img_last is None:
        frame_diff=cv2.absdiff(img_last,img_b)#--3
        cnts=cv2.findContours(frame_diff,
                                   cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)[0]
        #差分があった領域をファイルに出力--4
        for pt in cnts:
            x,y,w,h=cv2.boundingRect(pt)
            if w<100 or w>500: continue#ノイズを除去
            #抽出した領域を画像として保存
            imgex=frame[y:y+h,x:x+w]
            outfile=save_dir+"/"+str(no)+".jpg"
            cv2.imwrite(outfile,imgex)
            no+=1
    img_last=img_b
cap.release()
print("ok")

ok


In [13]:
import cv2
import os,glob,pickle
from sklearn.model_selection import train_test_split
from sklearn import datasets,metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pathlib import Path

#画像サイズやパスを指定
image_size=(64,32)
path=Path().resolve()
path_fish=path+'/fish'
path_nofish=path+'/nofish'
x=[]#画像データ
y=[]#ラベルデータ

#画像データを読み込んで配列に追加--1
def read_dir(path,label):
    files=glob.glob(path+"/*.jpg")
    for f in files:
        img=cv2.imread(f)
        img=cv2.resize(img,img_size)
        img_data=img.reshape(-1, )#一次元に展開
        x.append(img_data)
        y.append(label)
#画像データを読み込む
read_dir(path_nofish,0)
read_dir(path_fish,1)

#データを学習用とテスト用に分割する--2
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

#データを学習--3
clf=RandomForestClassifier()
clf.fit(x_train,y_train)

#精度の確認--4
y_pred=clf.predict(x_test)
print(accuracy_score(y_test,y_pred))

#データを保存
with open("fish.pkl","wb") as fp:
    pickle.dump(clf,fp)

TypeError: unsupported operand type(s) for +: 'WindowsPath' and 'str'

In [11]:
import cv2
import os, glob, pickle
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 画像の学習サイズやパスを指定
image_size = (64, 32)
path = os.path.dirname(os.path.abspath(__file__))
path_fish = path + '/fish'
path_nofish = path + '/nofish'
x = [] # 画像データ
y = [] # ラベルデータ

# 画像データを読み込んで配列に追加 --- (*1)
def read_dir(path, label):
    files = glob.glob(path + "/*.jpg")
    for f in files:
        img = cv2.imread(f)
        img = cv2.resize(img, image_size)
        img_data = img.reshape(-1, ) # 一次元に展開
        x.append(img_data)
        y.append(label)

# 画像データを読み込む
read_dir(path_nofish, 0)
read_dir(path_fish, 1)

# データを学習用とテスト用に分割する --- (*2)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# データを学習 --- (*3)
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

# 精度の確認 --- (*4)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

# データを保存 --- (*5)
with open("fish.pkl", "wb") as fp:
  pickle.dump(clf, fp)

NameError: name '__file__' is not defined